In [2]:
from backup.mongo_class import my_mongo
from binance import Client
from pandas import DataFrame, to_datetime, Timestamp
from datetime import datetime, timedelta
from tqdm import tqdm
from numpy import *
from scipy.stats import norm

In [3]:
mongo = my_mongo()

In [10]:
data = mongo.col_to_df('cex_trade', 'binance_kline_data', {'base_token': 'BTC'})

In [7]:
def cal_BS_option_price(S, K, T, r, b, sigma, iscall: bool):
    """

    :param S:
    :param K:
    :param T:
    :param r: risk——free
    :param b: dividend yield for crypto currency, b=0
    :param sigma:
    :param iscall:
    :return:
    """
    real_r = r - b
    d1 = (log(S/K) + (real_r + 0.5*sigma**2) * T)/(sigma*sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    if iscall:
        return S * norm.cdf(d1) - K * exp(-real_r * T) * norm.cdf(d2)
    else:
        return -S * norm.cdf(-d1) + K * exp(-real_r * T) * norm.cdf(-d2)

In [5]:
def cal_iv(P, S, K, T, r, b, iscall: bool, method: str='binary'):
    """

    :param P:
    :param S:
    :param K:
    :param T:
    :param r:
    :param b:
    :param iscall:
    :param method:
    :return:
    """
    if method == 'binary':
        sigma_up = 1
        sigma_down = 0.001
        sigma_mid = (sigma_up+sigma_down)/2
        while abs(cal_BS_option_price(S, K, T, r, b, sigma_mid, iscall) - P) > 0.0001:
            if cal_BS_option_price(S, K, T, r, b, sigma_down, iscall) < P < cal_BS_option_price(S, K, T, r, b, sigma_mid, iscall):
                sigma_up = sigma_mid
                sigma_mid = (sigma_up+sigma_down)/2
            else:
                sigma_down=sigma_mid
                sigma_mid=(sigma_up+sigma_down)/2
        return sigma_mid

    elif method == 'newton':
        pass


In [6]:
def cal_delta(S, K, sigma, r, b, T, iscall: bool):
    real_r = r - b
    d1 = (log(S/K) + (real_r + 0.5*sigma**2) * T)/(sigma*sqrt(T))
    if iscall:
        delta = norm.cdf(d1)
    else:
        delta = norm.cdf(d1) - 1
    return delta

In [8]:
def cal_gamma(S, K, sigma, r, b, T):
    real_r = r - b
    d1 = (log(S/K) + (real_r + 0.5*sigma**2) * T)/(sigma*sqrt(T))
    return exp(-d1**2 / 2)/(S * sigma * sqrt(2*pi*T))

In [ ]:
def cal_theta(S, K, sigma, r, b, T, iscall: bool):
    real_r = r - b
    d1 = (log(S/K) + (real_r + 0.5*sigma**2) * T)/(sigma*sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    theta_call = -(S * sigma * exp(-d1**2 / 2)) / (2 * sqrt(2*pi*T)) - real_r * K * exp(-real_r * T) * norm.cdf(d2)
    if iscall:
        theta = theta_call
    else:
        theta = theta_call + real_r * K * exp(-r*T)
    return theta

In [9]:
def cal_vega(S, K, r, b, T, sigma):
    real_r = r - b
    d1 = (log(S/K) + (real_r + 0.5*sigma**2) * T)/(sigma*sqrt(T))
    vega = S * norm.pdf(d1) * sqrt(real_r) / 100
    return vega

In [ ]:
expire_date = "2022-03-25 00:00:00"

In [23]:
to_datetime(expire_date) - to_datetime(data['start_time'][0]) / timedelta(days=1)

TypeError: unsupported operand type(s) for /: 'Timestamp' and 'datetime.timedelta'

In [25]:
Timestamp

TypeError: Cannot convert input [1 day, 0:00:00] of type <class 'datetime.timedelta'> to Timestamp

In [ ]:
cal_iv()